# Mencari kata yang sering muncul pada setiap pengguna
library yang digunakan `gensim`

In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer
from ast import literal_eval

import pandas as pd
import numpy as np
import json

## load data
konten akan dikelompokkan berdasarkan username, dan attribut yang diambil hanya `cleaned_content` dan `username`

In [81]:
data = pd.read_csv("../out/01-preprocessing.tsv", sep="\t")
data = data[['cleaned_content', 'username', 'mentioned', 'raw_content']]
data.groupby(by='username').count()

,cleaned_content,mentioned,raw_content
username,,,
00frhn,1,1,1
0899SH,1,1,1
0912NIKI,1,1,1
0tk0il,1,1,1
0verdoshit,1,1,1
...,...,...,...
zuzumbabi,3,3,3
zxdgiee,1,1,1
zys91,1,1,1


In [82]:
tfidf_vectorizer = TfidfVectorizer()

In [83]:
author_content = {}

In [87]:
for username, df in data.groupby(by='username'):
    if len(df) == 1:
        continue
    content = list(df['cleaned_content'].apply(literal_eval))
    mentioned = list(df['mentioned'].apply(literal_eval))
    raw_content = list(df['raw_content'])
    tfidf_matrix = tfidf_vectorizer.fit_transform([" ".join(c) for c in content])
    feature_names = tfidf_vectorizer.get_feature_names_out()
    
    result = []
    
    for i, doc in enumerate(content):
        feature_index = tfidf_matrix[i, :].nonzero()[1]
        tfidf_scores = zip(feature_index, [tfidf_matrix[i, x] for x in feature_index])
        c_doc = {
            'mentioned': mentioned[i],
            'raw_content': raw_content[i],
            'tf_idf': {}
        }
        for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
            c_doc['tf_idf'][w] = s
        result.append(c_doc)
    
    author_content[username] = result

In [92]:
with open('../out/02-tfidf.json', 'w') as f:
    json.dump(author_content, f, indent=4)